In [2]:
import logging
from typing import List

import pandas as pd
from langchain.chat_models import AzureChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

In [3]:
loader = TextLoader("../state_of_the_union.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

In [4]:
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': False}

hf_embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [5]:
hf_embeddings_vector_db = FAISS.from_documents(docs, hf_embeddings)
hf_embeddings_vector_db.save_local("faiss_index")

In [6]:
question = "What did the president say about economics?"
docs = hf_embeddings_vector_db.similarity_search(question)
docs[0].page_content

'We’re going after the criminals who stole billions in relief money meant for small businesses and millions of Americans.  \n\nAnd tonight, I’m announcing that the Justice Department will name a chief prosecutor for pandemic fraud. \n\nBy the end of this year, the deficit will be down to less than half what it was before I took office.  \n\nThe only president ever to cut the deficit by more than one trillion dollars in a single year. \n\nLowering your costs also means demanding more competition. \n\nI’m a capitalist, but capitalism without competition isn’t capitalism. \n\nIt’s exploitation—and it drives up prices. \n\nWhen corporations don’t have to compete, their profits go up, your prices go up, and small businesses and family farmers and ranchers go under. \n\nWe see it happening with ocean carriers moving goods in and out of America. \n\nDuring the pandemic, these foreign-owned companies raised prices by as much as 1,000% and made record profits.'

In [7]:
gpt35_azure_llm = AzureChatOpenAI(
    temperature=0,
    openai_api_key="02e3dbabaf334ccb959cbeadbd3f99c3",
    openai_api_base="https://llm-x-gpt.openai.azure.com/",
    deployment_name='LLM-X-GPT35-TURBO',
    openai_api_version="2023-03-15-preview"
)

In [12]:
gpt4_azure_llm = AzureChatOpenAI(
    temperature=0,
    openai_api_key="a8d69f68a36b40789df2cc3fdbaacda9",
    openai_api_base="https://llmx-gpt-canada-east.openai.azure.com/",
    deployment_name='LLM-X-GPT-4',
    openai_api_version="2023-03-15-preview"
)

In [9]:
from langchain.embeddings.openai import OpenAIEmbeddings

openai_azure_embeddings = OpenAIEmbeddings(
    openai_api_key="02e3dbabaf334ccb959cbeadbd3f99c3",
    openai_api_base="https://llm-x-gpt.openai.azure.com/",
    deployment_name='LLM-X-Embedding'
)

/home/watson/.virtualenvs/QnAEvaluation/lib/python3.9/site-packages/langchain/embeddings/openai.py:214: UserWarning: WARNING! deployment_name is not default parameter.
                    deployment_name was transferred to model_kwargs.
                    Please confirm that deployment_name is what you intended.
  warnings.warn(


In [13]:
logging.basicConfig()
logging.getLogger('langchain.retrievers.multi_query').setLevel(logging.INFO)
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=hf_embeddings_vector_db.as_retriever(),
    llm=gpt35_azure_llm
)

unique_docs = retriever_from_llm.get_relevant_documents(query=question)

INFO:langchain.retrievers.multi_query:Generated queries: ["1. Can you provide any information on the president's remarks regarding Zelensky?", "2. I'm curious to know what the president's comments were about Zelensky. Can you help me with that?", '3. Could you share any insights into what the president said specifically about Zelensky?']


In [14]:
from langchain.chains import RetrievalQA

qa_rag_chain = RetrievalQA.from_chain_type(
    llm=gpt4_azure_llm,
    retriever=hf_embeddings_vector_db.as_retriever()
)

question = "What did the president say about Zelensky?"
qa_rag_chain({"query": question})

{'query': 'What did the president say about Zelensky?',
 'result': "The president mentioned President Zelenskyy of Ukraine in the context of the Ukrainian people's resistance against Russia's invasion. He praised their fearlessness, courage, and determination, which he said inspires the world. However, he did not provide specific details about Zelenskyy's actions or policies."}

In [15]:
from langchain.chains import RetrievalQA

qa_rag_chain = RetrievalQA.from_chain_type(
    llm=gpt35_azure_llm,
    retriever=hf_embeddings_vector_db.as_retriever()
)

question = "What did the president say about Zelensky?"
qa_rag_chain({"query": question})

{'query': 'What did the president say about Zelensky?',
 'result': 'The president mentioned President Zelenskyy of Ukraine and praised the fearlessness, courage, and determination of the Ukrainian people.'}

In [16]:
from langchain.chains import QAGenerationChain
from langchain import PromptTemplate

templ = """You are a smart assistant designed to help high school teachers come up with reading comprehension questions.
Given a piece of text, you must come up with a {k} different question and answer pairs that can be used to test a student's reading comprehension abilities.
When coming up with this question/answer pair, each pair must be respond in the following format:

{{
    "question": "$YOUR_QUESTION_HERE",
    "answer": "$THE_ANSWER_HERE"
}}

So in your final answer you should response with a list of {k} pairs in this format:

```
[{{
    "question": "$YOUR_QUESTION_HERE",
    "answer": "$THE_ANSWER_HERE"
}},
 {{
    "question": "$YOUR_QUESTION_HERE",
    "answer": "$THE_ANSWER_HERE"
}},
 {{
    "question": "$YOUR_QUESTION_HERE",
    "answer": "$THE_ANSWER_HERE"
    }}
]
```

Please come up with a list of {k} question/answer pairs, in the specified list of JSONS format, for the following text:
----------------
{text}
"""

multi_qa_prompt = PromptTemplate.from_template(template=templ, partial_variables={"k": 5})
qa_generation_chain = QAGenerationChain.from_llm(llm=gpt35_azure_llm, prompt=multi_qa_prompt)

In [17]:
qna_GT = qa_generation_chain.run(docs[0].page_content)[0]
qna_GT

[{'question': 'What is the purpose of the chief prosecutor for pandemic fraud?',
  'answer': 'To go after the criminals who stole relief money meant for small businesses and Americans.'},
 {'question': 'How much will the deficit be reduced to by the end of this year?',
  'answer': 'Less than half of what it was before the current president took office.'},
 {'question': "What does the speaker mean by 'capitalism without competition isn't capitalism'?",
  'answer': 'The speaker believes that true capitalism requires competition, and without it, it becomes exploitation.'},
 {'question': "What happens to prices when corporations don't have to compete?",
  'answer': 'Prices go up.'},
 {'question': 'What did foreign-owned ocean carriers do during the pandemic?',
  'answer': 'They raised prices by as much as 1,000% and made record profits.'}]

In [18]:
questions_and_answers_llm = []
for i, qa in enumerate(qna_GT):
    question, answer = qa["question"], qa["answer"]
    llm_answer = qa_rag_chain({"query": question})["result"]
    questions_and_answers_llm.append({"question": question, "result": llm_answer})
    print(f"QA number {i + 1} \n")
    print(f"Question: {question}\n")
    print(f"Answer: {answer}\n")
    print(f"LLM Answer: {llm_answer}\n")
    print("--------------------------------------------------\n\n")

QA number 1 

Question: What is the purpose of the chief prosecutor for pandemic fraud?

Answer: To go after the criminals who stole relief money meant for small businesses and Americans.

LLM Answer: The purpose of the chief prosecutor for pandemic fraud is to go after the criminals who stole billions in relief money meant for small businesses and millions of Americans. They will be responsible for investigating and prosecuting cases of fraud related to the misuse of funds intended for pandemic relief.

--------------------------------------------------


QA number 2 

Question: How much will the deficit be reduced to by the end of this year?

Answer: Less than half of what it was before the current president took office.

LLM Answer: The given context does not provide specific information about the projected reduction of the deficit by the end of this year. Therefore, I don't have the information to answer your question.

--------------------------------------------------


QA number

In [25]:
def get_qna_with_chain_answers_df(qa_rag_chain, qna_GT) -> pd.DataFrame:
    qna_with_chain_answers = []

    for qna in qna_GT:
        question, answer = qna["question"], qna["answer"]
        chain_answer = qa_rag_chain({"query": question})["result"]
        qna_with_chain_answers.append({"question": question,
                                       "answer": answer,
                                       "chain_answer": chain_answer})

    qna_with_chain_answers_df = pd.DataFrame(qna_with_chain_answers)
    return qna_with_chain_answers_df

In [28]:
qna_with_chain_answers_df = get_qna_with_chain_answers_df(qa_rag_chain, qna_GT)

In [29]:
qna_with_chain_answers_df

,question,answer,chain_answer
0,What is the purpose of the chief prosecutor fo...,To go after the criminals who stole relief mon...,The purpose of the chief prosecutor for pandem...
1,How much will the deficit be reduced to by the...,Less than half of what it was before the curre...,The provided context does not mention a specif...
2,What does the speaker mean by 'capitalism with...,The speaker believes that true capitalism requ...,The speaker means that true capitalism require...
3,What happens to prices when corporations don't...,Prices go up.,"When corporations don't have to compete, their..."
4,What did foreign-owned ocean carriers do durin...,"They raised prices by as much as 1,000% and ma...","During the pandemic, foreign-owned ocean carri..."


In [36]:
from langchain.evaluation import load_evaluator, EvaluatorType
from langchain.evaluation import EmbeddingDistance

def get_grades_for_chain_qna(qna_with_chain_answers_df: pd.DataFrame) -> pd.DataFrame:
    grades_for_chain_qna = qna_with_chain_answers_df.copy()
    labeled_criteria_evaluator = load_evaluator(evaluator=EvaluatorType.LABELED_CRITERIA,
                                                criteria="correctness",
                                                llm=gpt35_azure_llm)

    pairwise_cosine_distance_evaluator = load_evaluator(evaluator=EvaluatorType.PAIRWISE_EMBEDDING_DISTANCE,
                                                        distance_metric=EmbeddingDistance.COSINE,
                                                        embeddings=hf_embeddings,
                                                        llm=gpt35_azure_llm)

    grades_for_chain_qna["labeled_criteria_grades"] = \
        qna_with_chain_answers_df.apply(
            lambda qna: labeled_criteria_evaluator.evaluate_strings(
                input=qna["question"],
                prediction=qna["chain_answer"],
                reference=qna["answer"])["score"], axis=1)

    grades_for_chain_qna["pairwise_cosine_distance_grade"] = \
        qna_with_chain_answers_df.apply(
            lambda qna: pairwise_cosine_distance_evaluator.evaluate_string_pairs(
                prediction=qna["answer"],
                prediction_b=qna["chain_answer"])["score"], axis=1)

    return grades_for_chain_qna

In [37]:
get_grades_for_chain_qna(qna_with_chain_answers_df)

,question,answer,chain_answer,labeled_criteria_grades,pairwise_cosine_distance_grade
0,What is the purpose of the chief prosecutor fo...,To go after the criminals who stole relief mon...,The purpose of the chief prosecutor for pandem...,1.0,0.432731
1,How much will the deficit be reduced to by the...,Less than half of what it was before the curre...,The provided context does not mention a specif...,NaN,0.686887
2,What does the speaker mean by 'capitalism with...,The speaker believes that true capitalism requ...,The speaker means that true capitalism require...,1.0,0.098474
3,What happens to prices when corporations don't...,Prices go up.,"When corporations don't have to compete, their...",1.0,0.463104
4,What did foreign-owned ocean carriers do durin...,"They raised prices by as much as 1,000% and ma...","During the pandemic, foreign-owned ocean carri...",NaN,0.242672
